<h1> Vaccine Design </h1>


This tutorial illustrates the use of epytope for population optimized epitope-based vaccines using OptiTope <a href="https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000246"> (Toussaint, et al. (2008) PLoS Comput Biol, 4(12))</a>

For this tutorial to work, you have to install a MIP solver that is supported by Pyomo like CPLEX, GLPK, or CBC. 

This tutorial will entail:
- Simple epitope prediction from a list of peptide sequences and protein sequences
- Simple epitope selection
- Activation/deactivation of optional constraints
- String-of-Beads design


In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from epytope.Core import Allele, Peptide, Protein,generate_peptides_from_proteins
from epytope.IO import read_lines, read_fasta
from epytope.EpitopePrediction import EpitopePredictorFactory
from epytope.EpitopeSelection import OptiTope
from epytope.EpitopeAssembly import EpitopeAssembly, EpitopeAssemblyWithSpacer

Using TensorFlow backend.


<h2> Chapter 1: Simple Epitope Prediction and preparation </h2>
    <br/>
Epitope selection is the most important step for vaccine design. It is concerned with selecting a small set of candidate epitopes to maximize the probability of inducing a long lasting and strong immune response. Epitope Selection is an interface to OptiTope, a highly flexible mathematical framework for epitope selection. OptiTope determines the provably optimal epitope set that maximizes the overall immunogenicity for a target population or a single person and user specified requirements.
 

We first start with reading in proteins from a FASTA file and constructing peptides from these. Then we have to construct a list of alleles to represent our target population and assign their probability of occurring in that targeted population.



In [2]:
proteins = read_fasta("./data/proteins.fasta", id_position=3, in_type=Protein)
peptides = generate_peptides_from_proteins(proteins,9)

alleles = []
with open("./data/allele_probabilities_europe.csv", "r") as f:
    for l in f:
        allele,prob = l.split()
        alleles.append(Allele(allele,prob=float(prob.strip())))

With that in place, we can start predicting the immunogenicity of each peptide HLA allele pair. Since the prediction of immunogenicity is an unsolved problem as of yet, the IC50 value of an epitope HLA allele pair is usually used as approximation. In this tutorial we use `BIMAS` as prediction engine. In addition, we can specify an allele specific binding threshold that is used internally to filter for specific constraints.


In [3]:
bimas = EpitopePredictorFactory("bimas")
supported_alleles = [allele for allele in filter(lambda a: a in bimas.supportedAlleles, alleles)]
epi_df =bimas.predict(peptides, alleles=supported_alleles)
threshold = {a.name:1 for a in supported_alleles}
epi_df.head()

Allele,HLA-B*15:01,HLA-B*44:03,HLA-A*03:01,HLA-A*11:01,HLA-A*01:01,HLA-B*08:01,HLA-C*04:01,HLA-C*07:02,HLA-B*27:05,HLA-A*68:01,...,HLA-B*51:01,HLA-B*51:02,HLA-A*24:02,HLA-B*35:01,HLA-B*52:01,HLA-B*40:06,HLA-B*40:01,HLA-B*39:02,HLA-B*39:01,HLA-B*07:02
Method,bimas,bimas,bimas,bimas,bimas,bimas,bimas,bimas,bimas,bimas,...,bimas,bimas,bimas,bimas,bimas,bimas,bimas,bimas,bimas,bimas
ScoreType,Score,Score,Score,Score,Score,Score,Score,Score,Score,Score,...,Score,Score,Score,Score,Score,Score,Score,Score,Score,Score
Peptides,,,,,,,,,,,,,,,,,,,,,
"(M, K, M, R, F, F, S, S, P)",0.002,0.002,0.00027,0.00004,0.000005,0.00020,0.0011,0.04,0.01,0.002,...,0.00260,0.0022,0.0018,0.001,0.0045,0.0020,0.002,0.120,0.02,0.003
"(K, M, R, F, F, S, S, P, C)",0.600,0.010,1.80000,0.00120,0.000500,0.10000,0.1000,0.40,15.00,0.010,...,0.05000,0.0500,0.2000,0.600,0.0495,0.0100,0.010,0.036,0.03,1.000
"(M, R, F, F, S, S, P, C, G)",0.001,0.001,0.00010,0.00004,0.001000,0.00010,0.0010,0.04,100.00,0.002,...,0.00572,0.0100,0.0010,0.001,0.0165,0.0030,0.001,0.010,0.10,0.001
"(R, F, F, S, S, P, C, G, K)",0.001,0.001,0.10000,1.20000,0.010000,0.00005,0.5000,0.02,15.00,0.600,...,0.01100,0.0500,0.1000,0.002,0.0500,0.0033,0.001,0.010,0.02,0.001
"(F, F, S, S, P, C, G, K, A)",0.010,0.015,0.00009,0.00200,0.000500,0.00100,5.5000,0.24,0.10,0.010,...,0.13000,0.4000,0.5500,0.010,0.1000,0.0500,0.022,0.100,0.10,0.015


<h2> Chapter 2: Epitope Selection </h2>

OptiTope defines the overall immunogenicity $I(E)$ of an epitope set $E$ as the sum over the immunogenicity of its components weighted by the HLA allele $a \in A$ frequencies $p_a$ of the target population $A$. This is a commonly made assumption due to a lack of understanding the interplay of different epitopes within a vaccine.

\begin{align}
I(E) = \sum_{e \in E}\sum_{a \in A} p_a \cdot i_{e,a}
\end{align}

Together with the prediction results, we are ready to use OptiTope. The simplest model OptiTope offers, is the optimal selections only constraint by the number of allowed selected epitopes:

\begin{align}
\max &\sum_{e \in E}x_e\sum_{a \in A}p_a\cdot i_{e,a}\\
\text{s.t.}\\
&\sum_{e \in E} x_e \le k
\end{align}


In [ ]:
optitope = OptiTope(epi_df, k=3, solver="cplex", threshold=threshold)
selected_epitope = optitope.solve()
selected_epitope

We can also activate additional constraints. For example we can restrict the selected epitope set to cover a certain percentage of all HLA alleles:

\begin{align}
\max &\sum_{e \in E}x_e\sum_{a \in A}p_a\cdot i_{e,a}\\
\text{s.t.}\\
&\sum_{e \in E}x_e \le k\\
&\sum_{e \in I_a} x_e \ge y_a~ ~ \forall a \in A \\
&\sum_{a \in A} y_a \ge \tau^{\text{HLA}}
\end{align}

In [ ]:
optitope.activate_allele_coverage_const(0.8)
optitope.solve()

and/or antigens:

\begin{align}
\max& \sum_{e \in E}x_e\sum_{a \in A}p_a\cdot i_{e,a}\\
\text{s.t.}\\
&\sum_{e \in E}x_e \le k\\
&\sum_{e \in I_a} x_e \ge y_a~ ~ \forall a \in A \\
&\sum_{a \in A} y_a \ge \tau^{\text{HLA}}\\
&\sum_{e \in I_g} x_e \ge z_g~ ~ \forall g \in G \\
&\sum_{g \in G} z_g \ge \tau^{\text{Antigen}}
\end{align}


In [ ]:
optitope.activate_allele_coverage_const(0.8)
optitope.activate_antigen_coverage_const(0.8)
optitope.solve()

you can also again deactivate these optional constraints with:

In [ ]:
optitope.deactivate_allele_coverage_const()
optitope.deactivate_antigen_coverage_const()
optitope.solve()

OptiTope  offers other optional constraints like epitope coverage or overlapping constraints. Also, the allele probability could be reused to represent for example the HLA expression of a individual, or can be completely ignored (which results in OptiTope to treat each allele uniformly for each locus).

<h2> Chapter 3: Epitope Assembly </h2><br/>
Epitope Assembly is concerned with ordering epitopes into a string-of-beads poly-peptide which maximizes the probability that the epitopes will be fully recovered after proteasomal cleavage. This is an important step for vaccine design and has potentially high impact on the efficacy of the designed vaccine. Epitope Assembly formulates the epitope ordering problem as a traveling salesperson problem where epitopes represent the cities to visit and the distances between the cities represent the recovery probabilities.

epytope offers two alternatives for string-of-beads design. A simple string-of-beads with `EpitopeAssembly.EpitopeAssembly`, and a string-of-beads design with additional spacer sequences connecting the epitopes (`EpitopeAssembly.EpitopeAssembly`).

Since these problems are formulated as Traveling salesperson optimization problem, the time to solve an instance can vary dramatically and solving to optimality can be impractical even for small instances. Therefore, epytope offers the possibility to approximate a string-of-beads configuration with the Lin-Kernighan traveling salesman heuristic besides an optimal solution. For that, the LKH implementation must be downloaded, compiled, and globally executable. The source code can be found at <a href=http://www.akira.ruc.dk/~keld/research/LKH/>http://www.akira.ruc.dk/~keld/research/LKH/</a>.

<h4> Simple String-of-Beads Design </h4><br/>
Lets start with the simple approach. For cleavage prediction we use `PCM` but any other cleavage site prediction method available in epytope can be used. To access the cleavage site prediction methods we use the `epytope.CleavagePrediction.CleavageSitePredictorFactory`, similar to the `EpitopePredictorFactory`. All we have to specify is a set of say through OptiTope selected epitopes and a cleavage site predictor. epytope will return a list of Peptides based on their optimal ordering.


In [ ]:
from epytope.CleavagePrediction import CleavageSitePredictorFactory

pcm = CleavageSitePredictorFactory("PCM")
SVB = EpitopeAssembly(selected_epitope,pcm,solver="cplex").solve()
print("SVB: ",  "-".join(map(str,SVB)))

<h4>String-of-Beads Design with Spacer Sequences</h4><br/>
Designing spacer sequences within a string-of-beads is more demanding and the model used is restricted to linear prediction function (currently only models derived from `APSSMEpitopePrediction`). 

Spacer Design for Epitope Assembly is concerned with determining the optimal amino acid composition and length of a small sequence connecting two epitopes within a string-of-beads poly-peptide and the epitope order to maximizes the probability that the epitopes will be fully recovered after proteasomal cleavage with the smallest neo-immunogenicity.
Hence, we have to specify a linear epitope predictor besides the cleavage site predictor and a HLA allele set for which the spacer sequences are optimized. Here again, epytope offers the possibility to approximate the solution via LKH.

We will use PCM as cleavage site and again BIMAS as epitope binding prediction methods and solve the design problem optimally.


In [ ]:
SVB_spacer = EpitopeAssemblyWithSpacer(selected_epitope, pcm, bimas, 
                                       alleles, threshold=threshold, solver="cplex").solve()
print "SVB with spacer: ", "-".join(map(str,SVB_spacer))